In [1]:
# testing reaction_recommender class
import collections
import itertools
import libsbml
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline  

script_path = "/Users/woosubs/Desktop/AutomateAnnotation/reaction_recommender/reaction_recommender/"
os.sys.path.append(script_path)

# import get_annotation as ga
from reaction_recommender import annotation_container as ac
from reaction_recommender import reaction_recommender as recommender

BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
BIOMD_12 = 'BIOMD0000000012.xml'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
# BioModel files
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
biomd_files = [f for f in os.listdir(BIOMODEL_DIR) if f[-4:] == ".xml"]
print(len(biomd_files))
biomd_fpaths = [os.path.join(BIOMODEL_DIR, one_file) for one_file in biomd_files]

# BiGG files
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")
bigg_files = [f for f in os.listdir(BIGG_DIR) if f[-4:]=='.xml']
bigg_fpaths = [os.path.join(BIGG_DIR, f) for f in bigg_files]
print(len(bigg_files))
one_fpath = bigg_fpaths[0]

1000
108


In [2]:
os.sys.path

['/Users/woosubs/Desktop/AutomateAnnotation/reaction_recommender/reaction_recommender/notebook',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python38.zip',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/lib-dynload',
 '',
 '/Users/woosubs/.local/lib/python3.8/site-packages',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/site-packages',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/site-packages/IPython/extensions',
 '/Users/woosubs/.ipython',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/site-packages/libsbml',
 '/Users/woosubs/Desktop/AutomateAnnotation/reaction_recommender/reaction_recommender/']

In [3]:
# Nov 14, 2021: using a shortened version, meaning H and D are removed
with open(os.path.join(DATA_DIR, 'all_shortened_chebi_to_formula_19nov2021.pkl'), 'rb') as f:
  ref_shortened_chebi_to_formula = pickle.load(f)
# below is full version
with open(os.path.join(DATA_DIR, 'all_possible_chebi_to_formula_12nov2021.pkl'), 'rb') as f:
  ref_full_chebi_to_formula = pickle.load(f)
# map secondary id to primary id
with open(os.path.join(DATA_DIR, 'chebi_second2prime_8nov2021.pickle'), 'rb') as f:
  ref_second2prime_dict = pickle.load(f)
# RHEA All Ids to Bi Id (KEGG Mappable)
with open(os.path.join(RHEA_DIR, 'rhea_all2bi.pkl'), 'rb') as f:
  ref_rhea2bi = pickle.load(f)
with open(os.path.join(RHEA_DIR, 'rhea_all2master.pkl'), 'rb') as f:
  ref_rhea2master = pickle.load(f)
with open(os.path.join(RHEA_DIR, 'rhea2chebi_reference.pkl'), 'rb') as f:
  ref_rhea2chebi = pickle.load(f)

In [4]:
one_container = ac.AnnotationContainer(model_file=os.path.join(BIGG_DIR, ecoli_fpath))

In [5]:
reader = libsbml.SBMLReader()
document = reader.readSBML(ecoli_fpath)
model = document.getModel()

In [6]:
one_spec_anot = model.getSpecies('M_glc__D_e').getAnnotationString()
one_spec_anot

'<sbml:annotation xmlns:sbml="http://www.sbml.org/sbml/level3/version1/core">\n  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n    <rdf:Description rdf:about="#M_glc__D_e">\n      <bqbiol:is xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/glc__D"/>\n          <rdf:li rdf:resource="http://identifiers.org/biocyc/META:Glucopyranose"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:12965"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:20999"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:4167"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:17634"/>\n          <rdf:li rdf:resource="http://identifiers.org/hmdb/HMDB00122"/>\n          <rdf:li rdf:resource="http://identifiers.org/hmdb/HMDB06564"/>\n          <rdf:li rdf:resource="http://identifiers.org/inchi_key/WQZGKKKJIJFFOK-GASJEMHNS

In [7]:
one_container.getOntologyFromString(one_spec_anot)

[('bigg.metabolite', 'glc__D'),
 ('biocyc', 'META:Glucopyranose'),
 ('chebi', 'CHEBI:12965'),
 ('chebi', 'CHEBI:20999'),
 ('chebi', 'CHEBI:4167'),
 ('chebi', 'CHEBI:17634'),
 ('hmdb', 'HMDB00122'),
 ('hmdb', 'HMDB06564'),
 ('inchi_key', 'WQZGKKKJIJFFOK-GASJEMHNSA-N'),
 ('kegg.compound', 'C00031'),
 ('kegg.drug', 'D00009'),
 ('metanetx.chemical', 'MNXM41'),
 ('sabiork', '1406'),
 ('sabiork', '1407'),
 ('seed.compound', 'cpd26821'),
 ('seed.compound', 'cpd00027')]

In [8]:
recom = recommender.ReactionRecommender(model_file=os.path.join(BIGG_DIR, ecoli_fpath))

In [9]:
recom.ref_mat

,C11O15P2R2,C37O7P,C11ClNO,C6N3O4,C105N7O36P2,C60NO7P,C5N4O4,(C5)n.C96H162N2O67P2,C6N4O4,C12N2,...,C16N2O11R,C2N2OR,C6Cl4O2,C19N5O7R2,C6N3,C12NO7PR,C15N5O5,C6N4O2R2,C24NO10S,C2O
RHEA:10003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RHEA:67147,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:67151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:67155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:67159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
recommender.ref_short_rhea2formula
dummy_dict = {'RHEA:10003': ['N', 'O', 'C5NO', 'C5O2'],
              'RHEA:10007': ['C8NS', 'C8NS']}
dummy_mat = recom.getReferenceMatrix(dummy_dict)
dummy_mat

,O,C8NS,C5O2,N,C5NO
RHEA:10003,1,0,1,1,1
RHEA:10007,0,1,0,0,0


In [11]:
dummy_mat.loc['RHEA:10003', 'N']

1

In [12]:
recom.getQueryMatrix(dummy_mat)

,O,C8NS,C5O2,N,C5NO
C11O15P2R2,0,0,0,0,0
C37O7P,0,0,0,0,0
C11ClNO,0,0,0,0,0
C6N3O4,0,0,0,0,0
C105N7O36P2,0,0,0,0,0
...,...,...,...,...,...
C12NO7PR,0,0,0,0,0
C15N5O5,0,0,0,0,0
C6N4O2R2,0,0,0,0,0
C24NO10S,0,0,0,0,0


In [15]:
res = recom.getCandidatesByReactionId()

In [17]:
len(res)

95

In [19]:
one_cand_dict = recom.getCandidatesByReactionId('R_PFK')
one_cand_dict

{'R_PFK': ['RHEA:12423',
  'RHEA:13380',
  'RHEA:14216',
  'RHEA:15656',
  'RHEA:16112']}

In [24]:
one_dict = {'R_PFK': ['RHEA:12423', 'RHEA:13380']}
recom.getCandidateReport(one_dict, df)

'Reaction R_PFK: M_atp_c + M_f6p_c = M_adp_c + M_fdp_c + M_h_c\nHas possible RHEA IDs as below:\n\n0. <RHEA:12420> ATP + D-tagatofuranose 6-phosphate = ADP + D-tagatofuranose 1,6-bisphosphate + H(+)\n1. <RHEA:13377> alpha-D-glucose 1-phosphate + ATP = ADP + alpha-D-glucose 1,6-bisphosphate + H(+)\n*********************************************************************************************************************\n'

In [18]:
from bioservices import ChEBI, Rhea
r = Rhea()
df = r.search("")
df = df.iloc[:, :2]
df.columns = ['id', 'equation']
df.set_index('id', inplace=True)
df.head()

,equation
id,
RHEA:10000,H2O + pentanamide = NH4(+) + pentanoate
RHEA:10004,benzyl isothiocyanate = benzyl thiocyanate
RHEA:10008,[protein]-dithiol + a hydroperoxide = [protein...
RHEA:10012,(R)-6-hydroxynicotine + H2O + O2 = 6-hydroxyps...
RHEA:10016,H2O + O-sinapoylcholine = choline + E-sinapate...


In [28]:
recom.sortCandidates(['RHEA:10015', 'RHEA:10003'])

['RHEA:10003', 'RHEA:10015']

In [40]:
one_flag = False
two_flag = False
for one_reac in model.getListOfReactions():
  for one_react in one_reac.getListOfReactants():
    if not one_flag and one_react.stoichiometry!=1.0:
      print("reaction: ", one_reac.getId())
      print(one_react.species)
      print(one_react.stoichiometry)
      one_flag = True
    if not two_flag and one_react.stoichiometry==1.0:
      print("reaction: ", one_reac.getId())
      print(one_react.species)
      print(one_react.stoichiometry)
      two_flag = True

reaction:  R_PFK
M_atp_c
1.0
reaction:  R_ATPS4r
M_h_e
4.0


In [54]:
one_reac = model.getReaction('R_ATPS4r')
min_num = 10
min_reaction = ''
for one_reac in model.getListOfReactions():
  num_ele = one_reac.getNumReactants() + one_reac.getNumProducts()
  if num_ele < min_num and num_ele>=2:
    min_num = num_ele
    min_reaction = one_reac.getId()
print(min_reaction)
print(min_num)

R_PGI
2


In [55]:
recom.getReactionString(model.getReaction('R_PGI'))

'M_g6p_c = M_f6p_c'

In [46]:
type(one_react)

libsbml.SpeciesReference

In [47]:
recom.formatElement(one_reac.getReactant('M_adp_c'))

'M_adp_c'

In [48]:
recom.formatElement(one_reac.getReactant('M_h_e'))

'4.0 * M_h_e'